In [1]:
# 連接到 google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


讀檔案

In [2]:
import datetime
import pandas as pd
starttime = datetime.datetime.now()
train = pd.read_csv('/content/drive/MyDrive/wsdm/train_ok.csv')
test = pd.read_csv('/content/drive/MyDrive/wsdm/test_ok.csv')
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

51


In [3]:
train = train.drop(['Unnamed: 0'],axis=1)
test = test.drop(['Unnamed: 0','Unnamed: 0.1','People global'],axis=1)

In [4]:
train.shape

(7377418, 55)

In [5]:
test.shape

(2556790, 55)

In [ ]:
train.head(2)

In [ ]:
test.head(2)

全部都跑 跑不動 刪掉相關程度較低的feature

In [8]:
# 刪除相關係數較低的feature
train = train.drop(['msno','language','time','notification','settings','Concert','My library_Search','Payment','Self profile more','topic-article-playlist'],axis=1)

In [9]:
# 刪除相關係數較低的feature
train = train.drop(['song_id','song_length','Discover Chart','Discover New','artist','my-daily-playlist','13','7','9'],axis=1)

In [10]:
train = train.drop(['city','bd','artist_name','explore',
          'Artist more','Discover Genre','Explore',
          'NS','Search','Search Home','Search Trends',
          'Unknown','song','song-based-playlist',
          'top-hits-for-artist','3','4'],axis=1)

In [ ]:
# train.columns

In [ ]:
train.shape

(7377418, 19)

In [ ]:
train.corr()

In [ ]:
train.head(2)

,discover,listen with,my library,radio,search,Album more,Discover Feature,Local playlist more,My library,Online playlist more,Others profile more,Radio,album,listen-with,local-library,local-playlist,online-playlist,radio.1,target
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1


test也要跟著改

In [11]:
test = test.drop(['msno','language','time','notification','settings','Concert','My library_Search','Payment','Self profile more','topic-article-playlist'],axis=1)

In [12]:
test = test.drop(['song_id','song_length','Discover Chart','Discover New','artist','my-daily-playlist','13','7','9'],axis=1)

In [13]:
test = test.drop(['city','bd','artist_name','explore',
          'Artist more','Discover Genre','Explore',
          'NS','Search','Search Home','Search Trends',
          'Unknown','song','song-based-playlist',
          'top-hits-for-artist','3','4'],axis=1)

In [ ]:
test.shape

(2556790, 19)

In [ ]:
test.head(2)

In [ ]:
# test_X = test.iloc[:,1:].values

建模

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = train.iloc[:,:-1].values
y = train.iloc[:,-1].values
test_X = test.iloc[:,1:].values

資料分割

In [15]:
# random_state固定比較好調整
# stratify=y，y才會分得比較平均
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state=1, stratify=y)

標準化

In [16]:
sc = StandardScaler()
sc.fit(X)
sc.fit(test_X)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

red:https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

LogisticRegression  (C=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from matplotlib.colors import ListedColormap

lr = LogisticRegression(C=1.0,random_state=1)
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# y_pred = lr.predict(X_test)
print('test accracy :%.3f' % lr.score(X_test,y_test))

test accracy :0.623


In [ ]:
target = lr.predict(test_X)
len(target)

2556790

In [ ]:
LR = test.iloc[:,0]
target = pd.DataFrame(target)

In [ ]:
LR = pd.concat([LR,target], axis=1)
LR

In [ ]:
LR['target'] = LR[0]

In [ ]:
LR = LR.drop([0],axis=1)

In [ ]:
# LR.shape
LR

In [ ]:
LR.isna().value_counts()

id     target
False  False     2556790
dtype: int64

In [ ]:
# LR
LR.to_csv('/content/drive/MyDrive/wsdm/LR2.csv',index=False)

LogisticRegression  (C=10)

In [ ]:
from sklearn.linear_model import LogisticRegression
from matplotlib.colors import ListedColormap
starttime = datetime.datetime.now()
lr = LogisticRegression(C=10.0,random_state=1)
lr.fit(X_train,y_train)
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

In [ ]:
# y_pred = lr.predict(X_test)
print('test accracy :%.3f' % lr.score(X_test,y_test))

accuracy_score:https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

KNN

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0).fit(X_train)
# y_km = km.fit_predict(X_train)

In [ ]:
y_kmeans_pred = km.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_kmeans_pred)

In [ ]:
target = km.predict(test_X)
len(target)

2556790

In [ ]:
KNN = test.iloc[:,0]
target = pd.DataFrame(target)

In [ ]:
KNN = pd.concat([KNN,target], axis=1)

In [ ]:
KNN

,id,0
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
2556785,2556785,0
2556786,2556786,0
2556787,2556787,0
2556788,2556788,0


In [ ]:
KNN['target'] = KNN[0]

In [ ]:
KNN = KNN.drop([0],axis=1)

In [ ]:
KNN

,id,target
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
2556785,2556785,0
2556786,2556786,0
2556787,2556787,0
2556788,2556788,0


In [ ]:
KNN.to_csv('/content/drive/MyDrive/wsdm/KNN.csv',index=False)

ref:https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

RandomForest 隨機森林

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=1)
forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
y_RFC_pred = forest.predict(X_test)

In [ ]:
accuracy_score(y_test, y_RFC_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy', n_estimators=20, random_state=1)
forest.fit(X_train,y_train)

In [ ]:
y_RFC_pred = forest.predict(X_test)

In [ ]:
accuracy_score(y_test, y_RFC_pred)

In [ ]:
target = forest.predict(test_X)
len(target)

2556790

In [ ]:
RFC = test.iloc[:,0]
target = pd.DataFrame(target)

In [ ]:
RFC = pd.concat([RFC,target], axis=1)

In [ ]:
# RFC

In [ ]:
RFC['target'] = RFC[0]

In [ ]:
RFC = RFC.drop([0],axis=1)

In [ ]:
# RFC

In [ ]:
RFC.to_csv('/content/drive/MyDrive/wsdm/RFC.csv',index=False)